## Importación de módulos

In [15]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

## Aplicación interactiva

In [ ]:

model = YOLO('yolov8n.pt')

video_path = "videos/traffic.mp4"
cap = cv2.VideoCapture(video_path)

track_history = defaultdict(lambda: [])

while cap.isOpened():

    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks and calculate speed
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y))) 
            if len(track) > 30: 
                track.pop(0)

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=5)

            # Calculate speed and infer direction based on the shape of the vehicle contour and tracking direction
            if len(track) >= 2:
                current_pos = np.array(track[-1])
                previous_pos = np.array(track[-2])
                speed = np.linalg.norm(current_pos - previous_pos)

                # Infer direction based on the shape of the vehicle contour and tracking direction
                contour = np.array([[int(x), int(y)], [int(x + w), int(y)], [int(x + w), int(y + h)], [int(x), int(y + h)]])
                rect = cv2.minAreaRect(contour)
                angle = rect[2]

                # Calculate the direction of movement of the rectangle
                rect_direction = "Este" if current_pos[0] > previous_pos[0] else "Oeste"

                if -45 <= angle < 45:
                    direction = "Este" if rect_direction == "Este" else "Oeste"
                else:
                    direction = "Norte" if current_pos[1] < previous_pos[1] else "Sur"

                # Display speed and direction on the frame
                cv2.putText(annotated_frame, f"Speed: {speed:.2f} px/frame", (int(x), int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.putText(annotated_frame, f"Direction: {direction}", (int(x), int(y) + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()